In [1]:
# https://huggingface.co/docs/transformers/main/en/tasks/masked_language_modeling

from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetMIDI, DataCollator
from transformers import DataCollatorForLanguageModeling
from miditok.utils import split_files_for_training
from torch.utils.data import DataLoader
from pathlib import Path
from random import shuffle

from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  Fil

In [9]:
# Load the tokenizer
prefix = '/media/maindisk/maximos/data/GiantMIDI-PIano/midis_v1.2/aug/midis'
saved_tokenizer_path = f'{prefix}_REMI_BPE_tokenizer.json'
path_to_dataset = prefix
path_to_train_splits = f'{prefix}_splits_REMI_BPE/train/midis'
path_to_valid_splits = f'{prefix}_splits_REMI_BPE/valid/midis'
path_to_tokenizer_pretrained = f'{prefix}_REMI_BPE_tokenizer'

max_seq_len = 1024

tokenizer = REMI(params=Path(saved_tokenizer_path))

tokenizer.save_pretrained(path_to_tokenizer_pretrained)

tokenizer.pad_token = tokenizer.special_tokens[0]
tokenizer.mask_token = tokenizer.special_tokens[1]

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15
)

In [21]:
tokenizer.pad_token = tokenizer.special_tokens[0]

In [22]:
files_paths = list(Path(path_to_dataset).glob("**/*.mid"))
shuffle(files_paths)

total_num_files = len(files_paths)
num_files_valid = round(total_num_files * 0.10)

midi_paths_valid = files_paths[:num_files_valid]
midi_paths_train = files_paths[num_files_valid:]

In [23]:
train_dataset = DatasetMIDI(
    files_paths=Path(path_to_train_splits).glob('**/*.mid'),
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
valid_dataset = DatasetMIDI(
    files_paths=Path(path_to_valid_splits).glob('**/*.mid'),
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
# collator = DataCollator(pad_token_id=tokenizer.pad_token_id, copy_inputs_as_labels=True)
# dataloader = DataLoader(dataset, batch_size=64, collate_fn=collator)


In [24]:
print(train_dataset[0])

{'input_ids': tensor([   2, 1312,  185,  ..., 2052, 8328,  217])}


In [25]:
model = AutoModelForMaskedLM.from_pretrained("distilbert/distilroberta-base")

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
training_args = TrainingArguments(
    output_dir='models/robertaGM',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

AttributeError: 'REMI' object has no attribute 'convert_tokens_to_ids'

In [5]:
from transformers import AutoTokenizer

In [10]:
# tokenizer_lang = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")
tokenizer_lang = AutoTokenizer.from_pretrained("/media/maindisk/maximos/data/GiantMIDI-PIano/midis_v1.2/aug/midis_REMI_BPE_tokenizer")

ValueError: Unrecognized model in /media/maindisk/maximos/data/GiantMIDI-PIano/midis_v1.2/aug/midis_REMI_BPE_tokenizer. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio-spectrogram-transformer, autoformer, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, encodec, encoder-decoder, ernie, ernie_m, esm, falcon, falcon_mamba, fastspeech2_con

In [32]:
tokenizer.get_special_tokens_mask = tokenizer_lang.get_special_tokens_mask

In [28]:
tokenizer.pad = tokenizer_lang.pad